## Exercise: Extract Ground Truth Visual Attributes from Segmented Images

In this exercise, you’ll extract **ground truth visual attributes**—including size, shape, color, and texture—for green peas captured during **fluidized bed drying**. This is the first critical step in training a machine learning model that can **predict visual properties of peas in real time**.

You are provided with:

* RGB images of green peas
* Corresponding segmentation masks
* A class, `VisualPropertiesExtractor`, that automates the extraction and calibration of object-level visual attributes

These images are divided into two categories:

* **Real-time images**: Captured directly inside the dryer
* **Offline images**: Captured after removing peas from the dryer

To ensure accurate ground truth measurements, each image set must be **calibrated** using a domain-specific calibrator (`realtime_calibrator` or `offline_calibrator`), which adjusts raw size and color values to match real-world metrics.

---

### Your Task

1. Load the real-time image and mask directories.

2. Initialize the `VisualPropertiesExtractor` with:

   * Calibration via `realtime_calibrator`
   * Group size of `3` images per time interval
   * Time step (`interval_per_image_group`) of `15` minutes
   * Output directory where results will be saved

3. Call `.process_data()` to generate:

   * Object-wise visual properties (JSON and CSV)
   * Mean visual properties per image
   * Group-wise time-aggregated visual summaries

4. Repeat steps 1–3 for the **offline image set**, using `offline_calibrator` and a separate output directory.

---

### Real-Time Calibration Example


In [11]:
from app.services.visual_attributes import VisualPropertiesExtractor, realtime_calibrator
from app.utils import dirs

In [12]:
# image and masks directories
image_dir = dirs.realtime_image_dir            # 'assets/realtime/images'
mask_dir = dirs.realtime_mask_dir              # 'assets/realtime/masks'

# Location where visual attribute results will be saved.
save_dir = dirs.realtime_visual_attr_dir       # 'assets/realtime/visual_attributes'

online_extractor = VisualPropertiesExtractor(
    start_image_index=1,
    images_directory=image_dir,
    masks_directory=mask_dir,
    image_mask_channels=(3, 1),
    exclude_partial_objects=False,
    cache_visual_attributes=False,
    calibrator=realtime_calibrator,
    image_group_size=3,                     # 3 images per drying interval
    interval_per_image_group=15,
    initial_group_interval=0,
    save_location=save_dir,
    overwrite_existing_record=True,
)

online_extractor.process_data()

#### Preview Results

##### a. Mean visual attributes for each image

In [13]:
online_extractor.mean_props_df

,image_id,time,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
0,img_1,0.0,0.442542,11.265594,12.308121,100.211627,38.395073,0.849738,61.804394,-29.570373,33.397433,120.629010,0.986691,0.235400,0.614488,5.989630,0.056063
1,img_2,0.0,0.357883,10.952287,11.680484,94.748160,37.104830,0.860431,61.465541,-27.272813,32.799658,136.543348,0.984997,0.246046,0.616916,5.923057,0.061249
2,img_3,0.0,0.495209,11.012389,12.402671,97.008253,38.868736,0.818407,59.725303,-28.488840,31.602536,106.057869,0.988426,0.242556,0.605043,6.051643,0.059835
3,img_4,15.0,0.553255,8.643716,9.705544,60.285234,29.600236,0.840873,57.504533,-18.230521,22.015440,169.548738,0.980823,0.256448,0.485929,6.032075,0.067425
4,img_5,15.0,0.455785,9.497305,10.372475,71.395272,32.302827,0.853705,55.203108,-19.205668,21.507067,136.054525,0.983192,0.237896,0.477199,6.213691,0.058323
5,img_6,15.0,0.434758,9.233401,10.089008,67.673493,31.343222,0.857614,57.609618,-19.073900,22.608091,150.417311,0.982307,0.242957,0.483752,6.101175,0.059888
6,img_7,30.0,0.514271,8.524408,9.641369,58.416163,29.865244,0.824105,54.670438,-14.712379,18.503243,173.722422,0.978241,0.248350,0.470877,6.118991,0.063538
7,img_8,30.0,0.489830,8.319758,9.274306,55.506230,28.474710,0.840972,57.067712,-15.056220,19.662941,183.594257,0.978292,0.238178,0.465746,6.138684,0.057986
8,img_9,30.0,0.515456,8.376760,9.444600,55.794303,28.764011,0.837394,57.307979,-14.774463,19.802037,178.427741,0.979694,0.246263,0.464217,6.143828,0.061900
9,img_10,45.0,0.493563,8.487995,9.566794,57.430207,29.512418,0.832681,56.953064,-13.803580,18.694211,174.307346,0.979759,0.248092,0.465098,6.126341,0.063995


##### b. Visual attributes grouped by time interval

In [14]:
online_extractor.grouped_props_df

,time,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
time,,,,,,,,,,,,,,,,
0.0,0.0,0.431878,11.076756,12.130425,97.322680,38.122880,0.842858,60.998413,-28.444009,32.599876,121.076742,0.986705,0.241334,0.612149,5.988110,0.059049
15.0,15.0,0.481266,9.124807,10.055676,66.451333,31.082095,0.850731,56.772419,-18.836697,22.043532,152.006858,0.982107,0.245767,0.482293,6.115647,0.061879
30.0,30.0,0.506519,8.406975,9.453425,56.572232,29.034655,0.834157,56.348710,-14.847687,19.322740,178.581473,0.978742,0.244263,0.466947,6.133834,0.061141
45.0,45.0,0.502370,8.358370,9.397977,55.661797,28.853155,0.833909,57.413695,-14.021502,19.230026,192.110841,0.978365,0.250347,0.466744,6.111327,0.064679
60.0,60.0,0.493739,8.189126,9.182139,53.603977,28.125864,0.835906,58.224588,-13.794031,19.779935,195.872844,0.977917,0.246926,0.476477,6.097093,0.062442


##### c. Visual attributes for individual objects in each image

In [15]:
online_extractor.comprehensive_props_df

,time,image_id,label,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,mean_intensity-0,mean_intensity-1,...,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
0,0,img_1,1,0.478238,11.154100,12.013314,97.80888,37.436312,143.989952,167.286644,...,0.877005,64.843330,-28.973719,36.147598,148.082167,0.985507,0.237086,0.602077,5.993129,0.056210
1,0,img_1,2,0.406701,12.011873,12.721647,113.43072,40.597800,146.279514,174.981846,...,0.864840,67.079541,-32.696187,37.738993,129.900703,0.987715,0.239650,0.601610,5.927160,0.057432
2,0,img_1,3,0.407118,11.316236,12.097473,100.67304,38.228337,131.599746,159.915548,...,0.865670,62.149885,-31.242477,33.018962,174.121937,0.981081,0.231429,0.605343,6.071159,0.053560
3,0,img_1,4,0.462712,10.840864,11.963393,92.39256,36.601061,144.898187,175.119576,...,0.866681,67.038261,-33.498642,36.962339,113.579829,0.988865,0.230561,0.619152,5.908417,0.053158
4,0,img_1,5,0.456072,11.805587,12.814779,109.56816,40.099568,154.921095,166.942679,...,0.856278,65.541541,-22.418304,33.937130,160.019232,0.983379,0.228093,0.614913,5.858227,0.052026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,60,img_15,20,0.553379,9.138968,9.957480,65.66040,31.403824,149.142077,152.866999,...,0.836659,61.711958,-16.023121,21.777383,199.082676,0.979747,0.268183,0.481760,5.894275,0.071922
356,60,img_15,21,0.417110,7.833747,8.690850,48.24456,27.046114,85.362478,84.446744,...,0.828798,41.119421,-10.983698,12.789402,117.254442,0.972218,0.210390,0.487008,6.220429,0.044264
357,60,img_15,22,0.560088,8.113959,9.272824,51.75768,27.878059,112.859967,114.213334,...,0.836873,50.206305,-12.796647,13.901116,128.476159,0.981510,0.262216,0.462451,6.130977,0.068757
358,60,img_15,23,0.473493,8.272594,9.157652,53.80128,28.119675,158.878798,155.951693,...,0.855031,63.190813,-13.383616,22.883461,157.930721,0.981714,0.207175,0.545218,6.003319,0.042921


---

### Offline Calibration Example

In [16]:
from app.services.visual_attributes import offline_calibrator

In [17]:
# Image and mask directory
image_dir = dirs.offline_image_dir             # 'assets/offline/images'
mask_dir = dirs.offline_mask_dir               # 'assets/offline/masks'

# Location where visual attribute results will be saved.
save_dir = dirs.offline_visual_attr_dir        # 'assets/offline/visual_attributes'

offline_extractor = VisualPropertiesExtractor(
    start_image_index=1,
    images_directory=image_dir,
    masks_directory=mask_dir,
    image_mask_channels=(3, 1),
    exclude_partial_objects=True,
    cache_visual_attributes=False,
    calibrator=offline_calibrator,
    image_group_size=1,                       # 1 image per drying interval
    interval_per_image_group=15,
    initial_group_interval=0,
    save_location=save_dir,
    overwrite_existing_record=True,
)

offline_extractor.process_data()

#### Preview results

##### a. Mean visual attributes for each image

In [18]:
offline_extractor.mean_props_df

,image_id,time,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
0,img_1,0.0,0.449060,7.179429,7.812685,41.075402,23.881147,0.892419,43.029015,-101.057490,42.518790,518.344229,0.953617,0.211819,0.366756,6.077017,0.045290
1,img_2,15.0,0.426258,6.413633,6.989915,32.889883,21.431825,0.890137,37.043231,-19.632611,30.950657,825.230670,0.921751,0.215403,0.279494,6.223836,0.047318
2,img_3,30.0,0.449697,6.052589,6.581821,29.031371,20.120468,0.893671,35.500282,-15.563911,29.286303,953.012941,0.906516,0.210112,0.269133,6.272911,0.044482
3,img_4,45.0,0.416376,5.769232,6.328513,26.707085,19.421547,0.885796,35.678718,-12.821440,28.901487,1024.268841,0.906297,0.215789,0.268508,6.311773,0.047092
4,img_5,60.0,0.356041,5.583593,5.984191,24.855893,18.496547,0.900069,36.557386,-11.469854,27.792793,1144.527755,0.898488,0.210101,0.258913,6.366196,0.044343
5,img_6,75.0,0.348206,5.542540,5.934944,24.576973,18.375319,0.898697,38.082744,-11.539623,28.375917,1214.129873,0.896110,0.210205,0.260513,6.356995,0.044317


##### b. Visual attributes grouped by time interval

In [19]:
offline_extractor.grouped_props_df

,time,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
time,,,,,,,,,,,,,,,,
0.0,0.0,0.449060,7.179429,7.812685,41.075402,23.881147,0.892419,43.029015,-101.057490,42.518790,518.344229,0.953617,0.211819,0.366756,6.077017,0.045290
15.0,15.0,0.426258,6.413633,6.989915,32.889883,21.431825,0.890137,37.043231,-19.632611,30.950657,825.230670,0.921751,0.215403,0.279494,6.223836,0.047318
30.0,30.0,0.449697,6.052589,6.581821,29.031371,20.120468,0.893671,35.500282,-15.563911,29.286303,953.012941,0.906516,0.210112,0.269133,6.272911,0.044482
45.0,45.0,0.416376,5.769232,6.328513,26.707085,19.421547,0.885796,35.678718,-12.821440,28.901487,1024.268841,0.906297,0.215789,0.268508,6.311773,0.047092
60.0,60.0,0.356041,5.583593,5.984191,24.855893,18.496547,0.900069,36.557386,-11.469854,27.792793,1144.527755,0.898488,0.210101,0.258913,6.366196,0.044343
75.0,75.0,0.348206,5.542540,5.934944,24.576973,18.375319,0.898697,38.082744,-11.539623,28.375917,1214.129873,0.896110,0.210205,0.260513,6.356995,0.044317


##### c. Visual attributes for individual objects in each image

In [20]:
offline_extractor.comprehensive_props_df

,time,image_id,label,eccentricity,equivalent_diameter,feret_diameter_max,filled_area,perimeter,mean_intensity-0,mean_intensity-1,...,roundness,L,a,b,contrast,correlation,energy,homogeneity,entropy,uniformity
0,0,img_1,1,0.570938,7.711543,8.759586,46.705976,25.920648,142.378124,175.659231,...,0.873556,41.327210,-86.998623,42.943472,468.670899,0.955721,0.199128,0.348188,6.320934,0.039652
1,0,img_1,2,0.345530,9.175597,9.666934,66.123913,30.218513,114.002088,153.145406,...,0.909960,30.543744,-171.130378,40.768773,213.533123,0.971929,0.199904,0.411461,6.185478,0.039961
2,0,img_1,3,0.541723,6.654719,7.303833,34.781580,22.033308,150.632263,179.967160,...,0.900326,43.757981,-47.947573,38.854389,571.978774,0.944680,0.171600,0.292703,6.316877,0.029447
3,0,img_1,4,0.409597,8.078939,8.603436,51.262351,27.113702,132.171890,165.785504,...,0.876255,36.794558,-88.950598,43.250505,351.286001,0.965626,0.241114,0.412349,6.081339,0.058136
4,0,img_1,5,0.479480,6.229250,6.902708,30.476246,20.872167,180.871911,212.821252,...,0.879096,58.504381,-148.663554,48.947222,1027.448701,0.926532,0.196704,0.356513,6.018571,0.038692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,75,img_6,72,0.495180,5.991235,6.491306,28.191782,20.078356,138.825022,147.529386,...,0.878772,31.124416,-10.053478,26.404070,1050.571429,0.901689,0.177843,0.225221,6.700894,0.031628
393,75,img_6,73,0.325381,4.825488,5.096085,18.288258,15.889448,154.902539,161.397392,...,0.910258,38.082362,-7.494396,20.313423,1227.391121,0.891697,0.209918,0.255907,6.303016,0.044066
394,75,img_6,74,0.358789,6.056236,6.526019,28.806830,20.209879,156.125926,162.748148,...,0.886294,38.449906,-9.212205,26.531383,899.962950,0.925599,0.196696,0.247701,6.562388,0.038689
395,75,img_6,75,0.346484,5.700069,6.022895,25.518207,18.862736,122.517954,131.655189,...,0.901261,23.283857,-9.662188,24.848378,1168.695418,0.868844,0.200745,0.238332,6.473665,0.040299


---

### Outcome

After completing this exercise, you will have:

* Clean and calibrated ground truth visual attributes for each pea image
* Time-series summaries of visual changes during drying
* A foundational dataset for training your ML model

Use the generated `.csv` or `.json` files for downstream ML tasks like regression or forecasting of visual quality indicators.

Great work preparing real-world training data!
